In [1]:
#|default_exp resample_img

# Resampling hyperspectral data

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import numpy as np
import pandas as pd
from spectral import *

In [12]:
#|export

class Resample():

  def __init__(self,
               path_to_spectrum : str , #Path to a CSV file containing spectral bands as float values stored as strings. Ensure that only columns with spectral data have digit-only names strings.
               bands_center : list , # list of band center of specific sensor
               fwhm : list , # 
               ):
          
          #read the dataframe
          init_df = pd.read_csv(path_to_spectrum)
          # filter to get only spectral data and change columns type to numeric
          df = init_df.loc[:, init_df.columns[init_df.columns.str.isdigit()]]
          df.columns=pd.to_numeric(df.columns)
          self.df = df
          #save the non spectral data
          self.non_spectral_data =init_df.loc[:, init_df.columns[~init_df.columns.str.isdigit()]]

  

  

In [14]:
instance = Resample(path_to_spectrum = "data\original\processesed\cleansamples_209.csv")

instance.df
# test.dtypes

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\USER\AppData\Local\Temp\ipykernel_10572\898297084.py:1: SyntaxWarning: invalid escape sequence '\o'
  instance = Resample(path_to_spectrum = "data\original\processesed\cleansamples_209.csv")


,350,351,352,353,354,355,356,357,358,359,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,0.089311,0.101862,0.116344,0.117707,0.113324,0.113424,0.115095,0.110248,0.103506,0.105503,...,0.448691,0.447846,0.446950,0.445301,0.444228,0.443859,0.443358,0.442132,0.440855,0.440254
1,0.071750,0.082479,0.097936,0.097098,0.090665,0.095351,0.098170,0.092902,0.085862,0.087585,...,0.382384,0.380523,0.379745,0.379368,0.378552,0.378313,0.376914,0.375947,0.374874,0.374323
2,0.041403,0.053078,0.069837,0.072832,0.069875,0.074908,0.074440,0.068492,0.064969,0.060439,...,0.202939,0.202970,0.201991,0.200577,0.199762,0.198509,0.197953,0.196985,0.196200,0.196456
3,0.127570,0.137622,0.152223,0.150098,0.142105,0.142880,0.146324,0.144206,0.141614,0.138125,...,0.410158,0.409710,0.408636,0.406409,0.405014,0.403905,0.403292,0.403064,0.402813,0.401774
4,0.027759,0.032114,0.052165,0.056613,0.052082,0.055811,0.054595,0.049068,0.046065,0.043011,...,0.304809,0.304953,0.304425,0.303253,0.302800,0.302721,0.302733,0.302398,0.301792,0.301727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.064846,0.054407,0.065575,0.061168,0.048756,0.047714,0.054091,0.057460,0.054480,0.052355,...,0.403771,0.402761,0.402004,0.401109,0.400511,0.400878,0.399908,0.399292,0.398573,0.397262
205,0.075402,0.060613,0.069765,0.064585,0.052161,0.051425,0.059872,0.064644,0.060609,0.055170,...,0.412809,0.412500,0.411685,0.410763,0.409443,0.407439,0.406662,0.405653,0.405583,0.405516
206,0.084860,0.069684,0.079296,0.073741,0.061434,0.062665,0.067750,0.070520,0.071387,0.067671,...,0.432248,0.431217,0.430221,0.428971,0.427898,0.427311,0.426792,0.426844,0.426273,0.424856
207,0.091414,0.080179,0.085475,0.078277,0.067148,0.068128,0.069800,0.072423,0.079867,0.076622,...,0.343658,0.343881,0.343307,0.342399,0.341766,0.340573,0.339903,0.339992,0.338872,0.338212


In [ ]:
#resampler_ven=BandResampler(df.columns.tolist(), center_ven, fwhm1=None, fwhm2=fwhm_ven)